In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html, dash_table, dcc, callback, Output, Input

In [2]:
#loading data
crs_data = pd.read_csv("crs_data.csv")

In [3]:
#understanding variables
#crs_data.head()
#crs_data.info()

In [4]:
#figuring out what the unique donor names are, to categorise
#unique_donor_name = crs_data["donor_name"].unique()
#print(unique_donor_name)
#unique_donors.to_csv('unique_donors_250517.csv', index=False)

In [5]:
#Grouping donors into mutually exclusive categories - G7, BRICS, UN, World Bank, Other - etc

g7 = ["United Kingdom", 
      "United States", 
      "France", 
      "Germany", 
      "Italy", 
      "Canada", 
      "Japan"]

brics = ["Brazil", 
         "Russia", 
         "India",
         "China", 
         "South Africa"] # no BRIC countries currently in dataset

eu_institutions = ["EU Institutions"]

foundation = ["Wellcome Trust",
            "IKEA Foundation",
            "William and Flora Hewlett Foundation",
            "Jacobs Foundation",
            "German Postcode Lottery",
            "Open Society Foundations",
            "UBS Optimus Foundation",
            "Susan T. Buffett Foundation",
            "World Diabetes Foundation",
            "Gates Foundation",
            "Rockefeller Foundation",
            "Laudes Foundation",
            "H&M Foundation",
            "David and Lucile Packard Foundation",
            "LEGO Foundation",
            "Children's Investment Fund Foundation",
            "Conrad N. Hilton Foundation",
            "Ford Foundation",
            "Good Ventures Foundation",
            "Citi Foundation",
            "Bloomberg Family Foundation",
            "Arcadia Fund"
            ]

un_agency = ["International Centre for Genetic Engineering and Biotechnology [ICGEB]",
            "WFP",
            "United Nations Industrial Development Organization [UNIDO]",
            "UNICEF",
            "UNFPA",
            "UNHCR",
            "UNDP",
            "UNAIDS",
            "UN Women",
            "International Labour Organisation [ILO]",
            "Joint Sustainable Development Goals Fund [Joint SDG Fund]",
            "UN Development Coordination Office",
            "Food and Agriculture Organisation [FAO]",
            "International Atomic Energy Agency [IAEA]",
            "Central Emergency Response Fund [CERF]",
            "IFAD",
            "COVID-19 Response and Recovery Multi-Partner Trust Fund [UN COVID-19 MPTF]"
            ]

world_bank = ["International Development Association [IDA]",
              "International Bank for Reconstruction and Development [IBRD]"]

other_dfi_ifi_bank = ["OPEC Fund for International Development [OPEC Fund]",
                "Global Fund",
                "Green Climate Fund [GCF]"
                "Private Infrastructure Development Group",
                "Global Environment Facility [GEF]",
                "Adaptation Fund",
                "Nordic Development Fund [NDF]",
                "Islamic Development Bank [IsDB]",
                "Asian Infrastructure Investment Bank [AIIB]",
                "Asian Development Bank [AsDB]",
                "IMF Concessional Trust Funds"
                ]

other_igo_ngo_ppp = ["World Health Organisation [WHO]",
                "World Trade Organisation",
                "WTO - International Trade Centre [ITC]",
                "WHO-Strategic Preparedness and Response Plan [SPRP]",
                "Global Alliance for Vaccines and Immunization [GAVI]"
                ]

other_country = ["Australia",
            "Austria",
            "Azerbaijan",
            "Belgium",
            "Bulgaria",
            "Croatia",
            "Czechia",
            "Denmark",
            "Estonia",
            "Finland",
            "Greece",
            "Hungary",
            "Iceland",
            "Ireland",
            "Israel",
            "Kazakhstan",
            "Korea",
            "Kuwait",
            "Latvia",
            "Liechtenstein",
            "Lithuania",
            "Luxembourg",
            "Malta",
            "Netherlands",
            "New Zealand",
            "Norway",
            "Poland",
            "Portugal",
            "Qatar",
            "Romania",
            "Saudi Arabia",
            "Slovak Republic",
            "Slovenia",
            "Spain",
            "Sweden",
            "Switzerland",
            "Türkiye",
            "Thailand",
            "United Arab Emirates"
]



In [6]:
# define function to apply donor grouping
def donor_group_function (donor):
    if donor in g7:
        return "G7"
    elif donor in brics:
        return "BRICS"
    elif donor in eu_institutions:
        return "EU Institutions"
    elif donor in foundation:
        return "Foundation"
    elif donor in un_agency:
        return "UN Agency"
    elif donor in world_bank:
        return "World Bank"
    elif donor in other_country:
        return "Other - Country"
    elif donor in other_dfi_ifi_bank:
        return "Other - DFI, IFI, Bank"
    else:
        return "Other - IGO, NGO, PPP"
    
#apply function crs_data to a new variable called 'donor group'
crs_data["donor_group"] = crs_data["donor_name"].apply(donor_group_function)

In [7]:
#grouping dataset, calculating sum and rounding to billion - for charts
crs_grouped = crs_data.groupby(['year','donor_group'], as_index=False)['value'].sum()
crs_grouped["value_bn"] = crs_grouped["value"] / 1000
crs_grouped["value_bn_rounded"] = crs_grouped["value_bn"].round(2)

crs_grouped.head()

,year,donor_group,value,value_bn,value_bn_rounded
0,2002,G7,4361.032566,4.361033,4.36
1,2002,Other - Country,131.898886,0.131899,0.13
2,2002,"Other - DFI, IFI, Bank",859.946212,0.859946,0.86
3,2002,UN Agency,39.043041,0.039043,0.04
4,2002,World Bank,2443.378006,2.443378,2.44


#### EXPLORING VISUALISATIONS WITH PLOTLY EXPRESS

In [22]:
fig = px.bar(
    crs_grouped,
    x="year",
    y="value_bn_rounded",
    color="donor_group",   # This separates the stacks by donor
    text_auto=True,       # Optional: shows values on bars
    title="ODA flows to Pakistan by year and donor group",
    labels={
        "year": "Year",
        "value_bn_rounded": "ODA disbursement ($billion)",
        "donor_group": "Donor Group"
    },
    color_discrete_sequence=px.colors.qualitative.Safe  # Optional: choose nice colors
)

# Customize the layout a bit
fig.update_layout(
    barmode="stack",  # Stack the bars
    title_font=dict(family="Arial", size=16),
    xaxis_title_font=dict(family="Arial", size=12),
    yaxis_title_font=dict(family="Arial", size=12),
    legend_title="Donor",
    template="plotly_white"
)

fig.show()

### definitely need to recategorise donor names, grouping roughly as follows:
# multilaterals: ADB, WB(IDA & IBRD), UN agencies, EU institution
# bilterals: USA, UK, France, Germany, Australia, Japan, Saudi, 'other'

CREATING A DASHBOARD - PLOTLY DASH

In [23]:
print(crs_grouped['donor_group'].unique())

['G7' 'Other - Country' 'Other - DFI, IFI, Bank' 'UN Agency' 'World Bank'
 'EU Institutions' 'Other - IGO, NGO, PPP' 'Foundation']


In [ ]:
# Initialize the app
app = Dash()

# App layout
app.layout = html.Div([
    html.Div(children='ODA Flows Dashboard'), 
    html.Hr(), #visual separator to break up sections of a page
    dcc.Dropdown(crs_grouped.donor_group.unique(),'G7', id='dropdown-selection'),  
    dcc.Graph(figure=fig, id='controls-and-graph')
])

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='dropdown-selection', component_property='value')
)
def update_graph(selected_group):
#filter for selected group first
    #crs_group_selected = pd.DataFrame(crs_grouped[crs_grouped['donor_group'] == selected_group])
#then draw the graph
    fig = px.bar(crs_grouped[selected_group], 
                 x='year', 
                 y='value_bn_round',
                 title = f'ODA by Year for {selected_group}')
    return [fig]

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\Users\helen\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:3805, in Index.get_loc(
    self=Index(['year', 'donor_group', 'value', 'value_bn', 'value_bn_rounded'], dtype='object'),
    key='G7'
)
   3804 try:
-> 3805     return self._engine.get_loc(casted_key)
        casted_key = 'G7'
        self = Index(['year', 'donor_group', 'value', 'value_bn', 'value_bn_rounded'], dtype='object')
   3806 except KeyError as err:

File index.pyx:167, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:196, in pandas._libs.index.IndexEngine.get_loc()

File pandas\\_libs\\hashtable_class_helper.pxi:7081, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas\\_libs\\hashtable_class_helper.pxi:7089, in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'G7'

The above exception was the direct cause of 